# Setup a computer with AiiDA 

In [ ]:
%%html
<img width="400px" src="http://scitas.epfl.ch/sites/all/docs/images/IMG_0198s.png"><br>
For more information visit <a target="_blank" href="http://scitas.epfl.ch/hardware/deneb-and-eltanin">epfl.ch</a>

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm import load_node, Code, Computer
from aiida.common.exceptions import NotExistent
from aiida.backends.utils import get_automatic_user, get_backend_type

if get_backend_type() == 'sqlalchemy':
    from aiida.backends.sqlalchemy.models.authinfo import DbAuthInfo
else:
    from aiida.backends.djsite.db.models import DbAuthInfo

import ipywidgets as ipw
from IPython.display import display, clear_output

from subprocess import check_call, check_output, call
from os import path
import re
import pexpect

In [ ]:
layout = ipw.Layout(width="400px")
style = {"description_width":"150px"}

## Step 1: Setup ssh

In [ ]:
def ssh_keygen():
    fn = path.expanduser("~/.ssh/id_rsa")
    if not path.exists(fn):
        print("Creating ssh key pair")
#        !  mkdir -p /project/.ssh
        ! ssh-keygen -f {fn} -t rsa -N ''
    
def is_host_known(hostname):
    fn = path.expanduser("~/.ssh/known_hosts")
    if not path.exists(fn):
        return False
    return call(["ssh-keygen", "-F", hostname]) == 0

def make_host_known(hostname, proxycmd=[]):
    fn = path.expanduser("~/.ssh/known_hosts")
    print("Adding keys from %s to %s"%(hostname, fn))
    hashes = check_output(proxycmd+["ssh-keyscan", "-H", hostname])
    with open(fn, "a") as f:
        f.write(hashes)

def can_login(hostname, username):
    userhost = username+"@"+hostname
    print("Trying ssh "+userhost+"... ", end='')
    ret = call(["ssh", userhost, "true"])
    print("Ok" if ret==0 else "Failed")
    return ret==0

def send_pubkey(hostname, username, password):
    inp_password.value = ""
    print("Setting up password-free connection to {}... ".format(hostname),end='')
    str_ssh = 'ssh-copy-id %s@%s' %(username, hostname)
    child = pexpect.spawn(str_ssh)
    index = child.expect(['s password:','WARNING: All keys were skipped',pexpect.EOF],timeout=10)
    if index == 0:
        child.sendline(password)
        try:
            child.expect("Now try logging into",timeout=5)
        except:
            print("Failed")
            raise Exception("Please check your username and/or password")
        print("Ok")
        child.close()
        return True
    elif index == 1:
        print ("Ok")
        print("Password-free access is already enabled")
        child.close()
        return True
    elif index == 2:
        print ('Failed')
        print(child.after, '\n', child.before)
        print("Are use sure the host address is correct?\n"
              "Host:{}".format(hostname))
        child.close()
        return False
    else:
        print ("Failed")
        print ("Unknown problem")
        print (child.before, child.after)
        return False

def is_in_config(hostname):
    fn = path.expanduser("~/.ssh/config")
    if not path.exists(fn):
        return False
    cfglines = open(fn).read().split("\n")
    return "Host "+hostname in cfglines

def write_ssh_config(targethost, username, proxyhost=None):
    fn = path.expanduser("~/.ssh/config")
    print("Adding section to "+fn)
    with open(fn, "a") as f:
        f.write("Host "+targethost+"\n")
        f.write("User "+username+"\n")
        if proxyhost is not None:
            f.write("ProxyCommand ssh "+username+"@"+proxyhost+" netcat %h %p\n")
        
def fetch_slurm_accounts(hostname, username, clustername):
    import re
    userhost = username+"@"+hostname
    print("Fetching slurm accounts for cluster {}".format(clustername))
    
    accounts_raw = check_output(["ssh", userhost, 
                             "sacctmgr", "show", "associations", "User={}".format(username),
                             "Cluster={}".format(clustername), "Format='Account'", "--noheader",
                             "--parsable"])
                             # "sinfo", "--noheader", "-o", "%P"])
    slurm_accounts = [_.split("|")[0].strip() for _ in accounts_raw.splitlines()]
    options = [("Please select a slurm account", False)]
    for l in slurm_accounts:
        # The second option (here None) is for partition/constraint
        options.append(["%s"%l, [l,None]])
    drop_account.options = options

def setup_ssh(proxyhost, targethost, username, clustername):
    ssh_keygen()
    
    # setup proxy first
    if not is_host_known(proxyhost):
        make_host_known(proxyhost)

    if not can_login(proxyhost, username):
        print("Please execute the following command and then try again:\n")
        print_pubkey(proxyhost, username)
        return False
                
    # now setup target host
    if not is_host_known(targethost):
        make_host_known(targethost, proxycmd=['ssh', username+"@"+proxyhost])
    
    if not is_in_config(targethost):
        write_ssh_config(targethost, username, proxyhost)
         
    # final check
    if can_login(targethost, username):
        fetch_slurm_accounts(targethost, username, clustername)
        print("Automatic ssh setup successful :-)")
        return True
    else:
        print("Automatic ssh setup failed, sorry :-(")
        return False
    
def setup_ssh_no_proxy(targethost, username, clustername, password):
    ssh_keygen()
    
    # now setup target host
    if not is_host_known(targethost):
        make_host_known(targethost)

    if not is_in_config(targethost):
        write_ssh_config(targethost, username)

    if not send_pubkey(targethost, username, password):
        return False

    # final check
    if can_login(targethost, username):
        fetch_slurm_accounts(targethost, username, clustername)
        print("Automatic ssh setup successful :-)")
        return True
    else:
        print("Automatic ssh setup failed, sorry :-(")
        return False

In [ ]:
def on_setup_ssh(b):
    with setup_ssh_out:
        clear_output()
        if len(inp_username.value.strip())==0:
            print("Please enter your EPFL username")
            return
        if len(inp_password.value.strip())==0:
            print("Please enter your password")
            return
        if drop_computer.value['status'] is False:
            print("Please specify the computer")
            return
        setup_ssh_no_proxy(drop_computer.value['host'], inp_username.value, drop_computer.value['name'], inp_password.value)

computers = [
    (   
        "Choose computer",
        {
            "status":False,
        }
    ),
    (
        "fidis",
        {
            "status": True,
            "host": "fidis.epfl.ch",
            "name": "fidis"
        }
    ),
    (
        "deneb",
        {
            "status": True,
            "host": "deneb1.epfl.ch",
            "name": "deneb"
        }
    ),
]
inp_username = ipw.Text(description="EPFL user name:", layout=layout, style=style)
inp_password = ipw.Password(description="Password:", layout=layout, style=style)
drop_computer = ipw.Dropdown(description="Slurm Account:", options=computers, style=style, layout=layout )
btn_setup_ssh = ipw.Button(description="Setup ssh")
btn_setup_ssh.on_click(on_setup_ssh)
setup_ssh_out = ipw.Output()


display(inp_username,inp_password,drop_computer,btn_setup_ssh, setup_ssh_out)

## Step 2: Setup AiiDA Computer

In [ ]:
print (inp_password.value)
inp_password.value = ""

In [ ]:
# https://github.com/aiidateam/aiida_core/blob/develop/aiida/cmdline/commands/computer.py#L400

def configure_computer(computer_name, username):
    # create DbAuthInfo
    authparams = {
        'compress': True,
        'gss_auth': False,
        'gss_deleg_creds': False,
        'gss_host': drop_computer.value['host'],
        'gss_kex': False,
        'key_policy': 'WarningPolicy',
        'load_system_host_keys': True,
        'port': 22,
        'timeout': 60,
        'username': inp_username.value,
    }
    aiidauser = get_automatic_user()
    authinfo = DbAuthInfo(dbcomputer=Computer.get(computer_name).dbcomputer, aiidauser=aiidauser)
    authinfo.set_auth_params(authparams)
    authinfo.save()
    
    ! verdi computer show {computer_name}    



def setup_computer(computer_name, username, account, partition):
    try:
        computer = Computer.get(computer_name)
        print("A computer called {} already exists.".format(computer_name))
        return
    except NotExistent:
        pass

    print("Creating new computer with name '{}'".format(computer_name))
    computer = Computer(name=computer_name)
    computer.set_hostname("deneb1.epfl.ch")
    computer.set_description("The Deneb Supercomputer at EPFL, Lausanne, Switzerland")
    computer.set_enabled_state(True)
    computer.set_transport_type("ssh")
    computer.set_scheduler_type("slurm")
    computer.set_workdir("/scratch/{username}/aiida_run_"+computer_name)
    # set_mpirun_command() must be called after set_scheduler_type()
    #cmd = "srun -n {tot_num_mpiprocs} -c $SLURM_CPUS_PER_TASK --cpu_bind=rank --hint=nomultithread"
    cmd = "srun -n {tot_num_mpiprocs}"
    computer.set_mpirun_command(cmd.split())
    ncpus = 24
    computer.set_default_mpiprocs_per_machine(ncpus)
    # Note: it would be better to use the custom scheduler commands, but they are not supported in Computer,
    # only in calc...
    prepend_text  = "### computer prepend_text start ###\n"
    prepend_text += "#SBATCH --account=%s\n"%account
    prepend_text += "### computer prepend_text end ###\n"
    computer.set_prepend_text(prepend_text)
    computer.store()
    
    configure_computer(computer_name, username)

In [ ]:
def on_account_change(c):
    if drop_account.value:
        inp_compname.value = inp_compname.value.rsplit("-",1)[0] + "-" + drop_account.value[0]

def on_setup_computer(b):
    with setup_comp_out:
        clear_output()
        if len(inp_compname.value.strip())==0:
            print("Please enter computer name")
            return
        if not drop_account.value:
            print("Plese select a slurm account")
            return
        account, partition = drop_account.value
        setup_computer(inp_compname.value, inp_username.value, account, partition)

    
inp_compname = ipw.Text(description="Computer name:", value="deneb", layout=layout, style=style)
drop_account = ipw.Dropdown(description="Slurm Account:", options={"first run ssh setup":False}, style=style, layout=layout )
drop_account.observe(on_account_change, names='value')
btn_setup_comp = ipw.Button(description="Setup Computer")
btn_setup_comp.on_click(on_setup_computer)
setup_comp_out = ipw.Output()
display(inp_compname, drop_account, btn_setup_comp, setup_comp_out)

## Step 3: Test Computer

In [ ]:
def on_test_computer(b):
    with test_out:
        clear_output()
        if len(inp_compname.value.strip())==0:
            print("Please enter computer name")
            return
        ! verdi computer test --traceback {inp_compname.value}

test_out = ipw.Output()
btn_test_comp = ipw.Button(description="Test Computer")
btn_test_comp.on_click(on_test_computer)
display(btn_test_comp, test_out)

## Step 4: Setup codes

In [ ]:
def setup_codes(computer_name, partition):
    computer = Computer.get(computer_name)
    
    qe_path = "/ssoft/spack/cornalin/v1/opt/spack/linux-rhel7-x86_E5v2_IntelIB/intel-17.0.2/espresso-6.1.0-qmy7wm3lqqyphqouvajxjzqbetoopakn/bin/"
    
    qe_version = "6.1"
    for codename in ['pw', 'ph', 'pp', 'projwfc', 'matdyn', 'q2r']:
        # QuantumESPRESSO code
        code = Code(remote_computer_exec=(computer, qe_path + "{}.x".format(codename)))
        code.label = "{}-{}".format(codename, qe_version)
        code.description = "Quantum ESPRESSO {}.x".format(codename)
        code.set_input_plugin_name("quantumespresso.{}".format(qe_version))
        prepend_text  = "### code prepend_text start ###\n"
        prepend_text += """module load intel
module load intel-mpi
module load intel-mkl
module load espresso/6.1.0-mpi
"""
        prepend_text += "### code prepend_text end ###\n"
        code.set_prepend_text(prepend_text)
        code.set_append_text("")
        code._reveal()
        code.store()
        full_string = "{}-{}@{}".format(codename, qe_version, computer_name)
        ! verdi code show "{full_string}"

In [ ]:
def on_setup_codes(b):
    with setup_code_out:
        clear_output()
        partition = drop_account.value[1]
        setup_codes(inp_compname.value, partition)

setup_code_out = ipw.Output()
btn_setup_codes = ipw.Button(description="Setup Codes")
btn_setup_codes.on_click(on_setup_codes)
display(btn_setup_codes, setup_code_out)